In [ ]:
import sys; sys.path.append('..')
import MeshFEM
import mesh, elastic_sheet, energy, benchmark
import triangulation
from tri_mesh_viewer import TriMeshViewer
import numpy as np

In [ ]:
V, E = mesh.load_raw('Data/victorinox.obj')
area = 3.0
V, F, edgeMarkers = triangulation.triangulate(V[:, 0:2], E, triArea=area, outputPointMarkers=False, outputEdgeMarkers=True)
m = mesh.Mesh(V, F)

isBoundary = np.zeros(m.numVertices(), dtype=np.bool)
isBoundary[m.boundaryVertices()] = True
creases = np.array([em for em in edgeMarkers if not isBoundary[em].all()])

In [ ]:
psi = energy.NeoHookeanYoungPoisson(2, 1, 0.3)
es = elastic_sheet.ElasticSheet(m, psi, creases)
es.thickness = 0.05
pinVars, pinVerts = es.prepareRigidMotionPins()
creaseVars = np.arange(es.numCreases()) + es.creaseAngleOffset()

In [ ]:
esview = TriMeshViewer(es, wireframe=True, width=1024, height=768)
esview.materialLibrary.material(False).color='#CC1111'
esview.show()

In [ ]:
import py_newton_optimizer
opts = py_newton_optimizer.NewtonOptimizerOptions()
opts.niter = 20

### Rerun the following cell to fold:

In [ ]:
es.setCreaseAngles(es.getCreaseAngles()[0] + 0.2 * np.pi / 16 * np.ones(es.numCreases()))
def iter_cb(prob, it):
    return
    if (it % 5 == 1):
       esview.update()
#benchmark.reset()
es.computeEquilibrium(loads=[], fixedVars=pinVars + list(creaseVars), cb=iter_cb, opts=opts)
esview.update()
#benchmark.report()